In [ ]:
# step1
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

# step2
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# # step3
# wd = webdriver.Chrome('chromedriver',options=options)

import sys, os
from selenium import webdriver
from pandas import DataFrame
from bs4 import BeautifulSoup
from datetime import datetime
import time
from tqdm import tqdm
import csv

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [80.8 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:11 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [946 kB]
Get:13 http

## DataFrame에 저장 후 CSV 파일에 저장

In [ ]:
df = []
col = ['date','title','content','url']
df = DataFrame(df, columns=col)

df

In [ ]:
for page in tqdm(range(1,401)):
  query = input('검색어를 입력하세요')
  driver = webdriver.Chrome('chromedriver',options=options)
  driver.get('https://www.chosun.com/nsearch/?query={}&page={}&siteid=&sort=1&date_period=all&date_start=&date_end=&writer=&field=&emd_word=&expt_word=&opt_chk=false&app_check=0&website=www,chosun&category='.format(query, page))
  driver.implicitly_wait(10)
  html = driver.page_source
  soup = BeautifulSoup(html)

  urlList = soup.find_all('a', {'class' : 'text__link story-card__headline | box--margin-none text--black font--primary h3 text--left'})
  dateList = soup.find_all('div', {'class': 'story-card__breadcrumb | text--grey-60 font--primary font--size-sm-14 font--size-md-14 box--margin-bottom-sm text--line-height-1.43'})

  for i in range(0,20,2):
    news = {'url':urlList[i]['href'], "date": dateList[i].text.split('| ')[2], 'title':urlList[i].text, "content": ""}
    df = df.append(news, ignore_index=True) 
  print('{}페이지 완료'.format(page))

In [ ]:
df

In [ ]:
df.to_csv('chosun.csv', encoding='utf-8-sig')

## CSV 파일에 바로 저장

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import urllib.request

driver = webdriver.Chrome('chromedriver',options=options)
query = input('검색어를 입력하세요')
with open('joongang.csv', 'w') as file:
    wr = csv.writer(file)
    wr.writerow(['time', 'title', 'content','url'])
    for page in tqdm(range(1,401)):
      for i in range(1,11):
        title_text = ""
        date_text = ""
        content_text = ""
        url = 'https://www.chosun.com/nsearch/?query={}&page={}&siteid=&sort=1&date_period=all&date_start=&date_end=&writer=&field=&emd_word=&expt_word=&opt_chk=false&app_check=0&website=www,chosun&category='.format(query, page)
        driver.get(url)
        table = driver.find_element_by_css_selector('#main > div.search-feed')
        try:
          try:
            title = driver.find_element_by_css_selector('#main > div.search-feed > div:nth-child('+str(i)+') > div > div.story-card.story-card--art-left.\|.flex.flex--wrap.box--hidden-sm > div.story-card-right.\|.grid__col--sm-8.grid__col--md-8.grid__col--lg-8.box--pad-left-xs > div.story-card__headline-container.\|.box--margin-bottom-xs > div > a')
          except:
            title = driver.find_element_by_css_selector('#main > div.search-feed > div:nth-child('+str(i)+') > div > div.story-card.story-card--art-left.\|.flex.flex--wrap.box--hidden-sm > div.story-card-right.\|.grid__col--sm-12.grid__col--md-12.grid__col--lg-12.box--pad-left-xs > div.story-card__headline-container.\|.box--margin-bottom-xs > div > a')
          date = driver.find_element_by_css_selector('#main > div.search-feed > div:nth-child('+str(i)+') > div > div.story-card.story-card--art-left.\|.flex.flex--wrap.box--hidden-sm > div.story-card-right.\|.grid__col--sm-8.grid__col--md-8.grid__col--lg-8.box--pad-left-xs > div.story-card__breadcrumb.\|.text--grey-60.font--primary.font--size-sm-14.font--size-md-14.box--margin-bottom-sm.text--line-height-1\.43 > span:nth-child(3)')
          url = title.get_attribute('href')
          title_text = title.text
          date_text = date.text
          print(title_text)
          driver.implicitly_wait(1)
          driver.get(url)
          try: 
            content = driver.find_element_by_css_selector('#fusion-app > div.article > div:nth-child(2) > div > section > article > section')
            content_text = content.text 
          except:
            pass
        except:
          pass
        wr.writerow([date_text, title_text, content_text, url])
      print('{}page 완료'.format(page))
